In [2]:
from datetime import datetime
from pathlib import Path

import torch

from converter import SchematicArrayConverter
from modules import LightningTransformerMinecraftStructureGenerator

model_version = 47
output_dir = 'schematic_viewer/public/schematics/'
checkpoint_path = f'lightning_logs/version_{model_version}/checkpoints/last.ckpt'
model = LightningTransformerMinecraftStructureGenerator.load_from_checkpoint(
    checkpoint_path)
model.eval()

converter = SchematicArrayConverter()

c:\Users\mmmfr\Documents\Repositories\minecraft-schematic-generator\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os

from common.file_paths import BASE_DIR
from converter import SchematicArrayConverter
from modules.data_module import MinecraftDataModule

hdf5_file = os.path.join(BASE_DIR, 'data.h5')
data_module = MinecraftDataModule(
    file_path=hdf5_file,
    batch_size=192,
    # num_workers=4
)

data_module.setup()
val_dataloaders = data_module.val_dataloader()
schematic_array_converter = SchematicArrayConverter()

In [4]:
import random

# Pick a random validation dataloader
dataloader = random.choice(val_dataloaders)
i = val_dataloaders.index(dataloader)
dataset_name = data_module.val_datasets[i][0]
trees_pos_1_dataset = data_module.val_datasets[i][1]

# dataset_name = 'trees_pos_1'
# trees_pos_1_dataset = None
# for name, dataset in data_module.val_datasets:
#     if name == dataset_name:
#         trees_pos_1_dataset = dataset
#         break

# Pick a random sample from the dataloader
i = random.randint(0, len(trees_pos_1_dataset) - 1)
masked_structure, full_structure = trees_pos_1_dataset[i]
masked_structure = masked_structure.to(model.device)
full_structure = full_structure.to(model.device)

# Convert the sample to the desired format using the provided function
masked_structure_schematic = schematic_array_converter.array_to_schematic(
    masked_structure)
masked_structure_schematic.name = 'Test'

# Save the sample
filename = f'sample_epoch_{random.random()}_dataloader_{dataset_name}.schem'
filepath = os.path.join('schematic_viewer/public/schematics/masked/', filename)
masked_structure_schematic.save_to_file(Path(filepath), 2)

filled_structure = masked_structure.clone()

In [5]:
import time

filled_structure = masked_structure.clone()
filled_structure[filled_structure == 1] = 0

# Generate a sample using the model
start_time = time.time()
model.complete_structure(filled_structure, temperature=1.0, fill_order='random')
end_time = time.time()
print(f'Generation time: {end_time - start_time}')

# Convert the sample to the desired format using the provided function
filled_structure_schematic = schematic_array_converter.array_to_schematic(
    filled_structure)
filled_structure_schematic.name = 'Test'

# Save the sample
filename = f'sample_epoch_{random.random()}_dataloader_{dataset_name}.schem'
filepath = os.path.join('schematic_viewer/public/schematics/filled/', filename)
filled_structure_schematic.save_to_file(Path(filepath), 2)

Generation time: 3.169405698776245


In [18]:
import time
import json
import requests

filled_structure = masked_structure.clone()
filled_structure[filled_structure == 1] = 0

# The URL of the FastAPI streaming endpoint
url = 'http://127.0.0.1:8000/complete-structure/'

# The input data to send to the server
input_data = {
    'temperature': 0.7,
    'structure': filled_structure.tolist()
}

serialized_data = json.dumps(input_data)

start_time = time.time()

# Make a POST request and stream the response
response = requests.post(url, data=serialized_data, stream=True)

# Check if the request was successful
if response.status_code == 200:
    # Iterate over the response
    for line in response.iter_lines():
        # Filter out keep-alive new lines
        if line:
            decoded_line = line.decode('utf-8')
            json_data = json.loads(decoded_line)
            value = json_data['value']
            z, y, x = json_data['position']
            filled_structure[z, y, x] = value
else:
    print(f"Error: {response.status_code}")

end_time = time.time()
print(f'Request time: {end_time - start_time}')

# Convert the sample to the desired format using the provided function
filled_structure_schematic = schematic_array_converter.array_to_schematic(
    filled_structure)
filled_structure_schematic.name = 'Test'

# Save the sample
filename = f'sample_epoch_{random.random()}_dataloader_{dataset_name}.schem'
filepath = os.path.join('schematic_viewer/public/schematics/filled/', filename)
filled_structure_schematic.save_to_file(Path(filepath), 2)

Request time: 6.34930682182312
